In [1]:
!pip install paddlepaddle==1.6.3 -i https://mirror.baidu.com/pypi/simple
!pip install parl==1.3.1 -i https://mirror.baidu.com/pypi/simple
!pip install rlschool==0.3.1 -i https://mirror.baidu.com/pypi/simple

Looking in indexes: https://mirror.baidu.com/pypi/simple
Looking in indexes: https://mirror.baidu.com/pypi/simple
Looking in indexes: https://mirror.baidu.com/pypi/simple
     |████████████████████████████████| 461 kB 5.8 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 18.6 MB/s eta 0:00:01
     |████████████████████████████████| 614 kB 15.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 11.7 MB/s eta 0:00:01
ERROR: gym 0.14.0 has requirement pyglet<=1.3.2,>=1.2.0, but you'll have pyglet 1.5.0 which is incompatible.
  Attempting uninstall: Pillow
    Found existing installation: Pillow 5.4.1
    Uninstalling Pillow-5.4.1:
      Successfully uninstalled Pillow-5.4.1
  Attempting uninstall: pyglet
    Found existing installation: pyglet 1.3.2
    Uninstalling pyglet-1.3.2:
      Successfully uninstalled pyglet-1.3.2


In [1]:
# 检查依赖包版本是否正确
!pip list | grep paddlepaddle
!pip list | grep parl
!pip list | grep rlschool

paddlepaddle                       1.6.3              
parl                               1.3.1              
rlschool                           0.3.1              


In [2]:
import os
import numpy as np
import gym

import parl
from parl import layers
from paddle import fluid
from parl.utils import logger
from parl.utils import action_mapping # 将神经网络输出映射到对应的 实际动作取值范围 内
from parl.utils import ReplayMemory # 经验回放

from rlschool import make_env  # 使用 RLSchool 创建飞行器环境

/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/anaconda3/lib/python3.7/site-packages/pandas/core/tools/datetimes.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/anaconda3/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/anaconda3/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs fro

In [3]:
######################################################################
######################################################################
#
# 1. 请设定 learning rate，尝试增减查看效果
#
######################################################################
######################################################################
ACTOR_LR = 0.0002   # Actor网络更新的 learning rate
CRITIC_LR = 0.001   # Critic网络更新的 learning rate

GAMMA = 0.99        # reward 的衰减因子，一般取 0.9 到 0.999 不等
TAU = 0.001         # target_model 跟 model 同步参数 的 软更新参数
MEMORY_SIZE = 1e6   # replay memory的大小，越大越占用内存
MEMORY_WARMUP_SIZE = 1e4      # replay_memory 里需要预存一些经验数据，再从里面sample一个batch的经验让agent去learn
REWARD_SCALE = 0.01       # reward 的缩放因子
BATCH_SIZE = 256          # 每次给agent learn的数据数量，从replay memory随机里sample一批数据出来
TRAIN_TOTAL_STEPS = 1e6   # 总训练步数
TEST_EVERY_STEPS = 1e4    # 每个N步评估一下算法效果，每次评估5个episode求平均reward

In [4]:
class PolicyModel(parl.Model):
    def __init__(self, act_dim, init_logvar=-1):
        ######################################################################
        ######################################################################
        #
        # 2. 请配置model结构
        #
        ######################################################################
        ######################################################################
        self.act_dim = act_dim
        self.fc1 = layers.fc(size=100, act='relu')
        self.fc2 = layers.fc(size=act_dim, act='tanh')
        
        self.logvars = layers.create_parameter(
            shape=[act_dim],
            dtype='float32',
            default_initializer=fluid.initializer.ConstantInitializer(
                init_logvar))

    def policy(self, obs):
        ######################################################################
        ######################################################################
        #
        # 3. 请组装policy网络
        #
        ######################################################################
        ######################################################################
        hid1 = self.fc1(obs)
        means = self.fc2(hid1)
        logvars = self.logvars()
        return means, logvars

    
    def sample(self, obs):
        means, logvars = self.policy(obs)
        print(means, logvars)
        print('aaaaa')
        sampled_act = means + (
            layers.exp(logvars / 2.0) *  # stddev
            layers.gaussian_random(shape=(self.act_dim, ), dtype='float32'))
        return sampled_act

In [5]:
class ValueModel(parl.Model):
    def __init__(self):
        ######################################################################
        ######################################################################
        #
        # 4. 请配置model结构
        #
        ######################################################################
        ######################################################################
        self.fc1 = layers.fc(size=100, act='relu')
        self.fc2 = layers.fc(size=1, act=None)

class ValueModel(parl.Model):
    def __init__(self, obs_dim, act_dim):
        super(ValueModel, self).__init__()
        hid1_size = obs_dim * 10
        hid3_size = 5
        hid2_size = int(np.sqrt(hid1_size * hid3_size))

        self.lr = 1e-2 / np.sqrt(hid2_size)

        self.fc1 = layers.fc(size=hid1_size, act='tanh')
        self.fc2 = layers.fc(size=1)

    def value(self, obs):
        hid1 = self.fc1(obs)
        V = self.fc2(hid1)
        V = layers.squeeze(V, axes=[])
        return V

In [6]:
    
class BipedalWalkerModel(parl.Model):
    def __init__(self, obs_dim, act_dim, init_logvar=-1.0):
        self.policy_model = PolicyModel(act_dim, init_logvar)
        self.value_model = ValueModel(obs_dim, act_dim)
#         self.policy_lr = self.policy_model.lr
#         self.value_lr = self.value_model.lr

    def policy(self, obs):
        return self.policy_model.policy(obs)

    def policy_sample(self, obs):
        return self.policy_model.sample(obs)

    def value(self, obs):
        return self.value_model.value(obs)

In [7]:
from parl.algorithms import PPO

In [8]:
class BiPedalwalkerAgent(parl.Agent):
    def __init__(self,
                 algorithm,
                 obs_dim,
                 act_dim,
                 kl_targ,
                 loss_type,
                 beta=1.0,
                 epsilon=0.2,
                 policy_learn_times=20,
                 value_learn_times=10,
                 value_batch_size=256):
        self.alg = algorithm
        self.obs_dim = obs_dim
        self.act_dim = act_dim
        assert loss_type == 'CLIP' or loss_type == 'KLPEN'
        self.loss_type = loss_type
        super(BiPedalwalkerAgent, self).__init__(algorithm)

        self.policy_learn_times = policy_learn_times
        # Adaptive kl penalty coefficient
        self.beta = beta
        self.kl_targ = kl_targ

        self.value_learn_times = value_learn_times
        self.value_batch_size = value_batch_size
        self.value_learn_buffer = None

    def build_program(self):
        self.policy_predict_program = fluid.Program()
        self.policy_sample_program = fluid.Program()
        self.policy_learn_program = fluid.Program()
        self.value_predict_program = fluid.Program()
        self.value_learn_program = fluid.Program()

        with fluid.program_guard(self.policy_sample_program):
            obs = layers.data(
                name='obs', shape=[self.obs_dim], dtype='float32')
            print(obs)
            sampled_act = self.alg.sample(obs)
            self.policy_sample_output = [sampled_act]

        with fluid.program_guard(self.policy_predict_program):
            obs = layers.data(
                name='obs', shape=[self.obs_dim], dtype='float32')
            means = self.alg.predict(obs)
            self.policy_predict_output = [means]

        with fluid.program_guard(self.policy_learn_program):
            obs = layers.data(
                name='obs', shape=[self.obs_dim], dtype='float32')
            actions = layers.data(
                name='actions', shape=[self.act_dim], dtype='float32')
            advantages = layers.data(
                name='advantages', shape=[1], dtype='float32')
            if self.loss_type == 'KLPEN':
                beta = layers.data(name='beta', shape=[], dtype='float32')
                loss, kl = self.alg.policy_learn(obs, actions, advantages,
                                                 beta)
            else:
                loss, kl = self.alg.policy_learn(obs, actions, advantages)

            self.policy_learn_output = [loss, kl]

        with fluid.program_guard(self.value_predict_program):
            obs = layers.data(
                name='obs', shape=[self.obs_dim], dtype='float32')
            value = self.alg.value_predict(obs)
            self.value_predict_output = [value]

        with fluid.program_guard(self.value_learn_program):
            obs = layers.data(
                name='obs', shape=[self.obs_dim], dtype='float32')
            val = layers.data(name='val', shape=[], dtype='float32')
            value_loss = self.alg.value_learn(obs, val)
            self.value_learn_output = [value_loss]

    def policy_sample(self, obs):
        feed = {'obs': obs}
        sampled_act = self.fluid_executor.run(
            self.policy_sample_program,
            feed=feed,
            fetch_list=self.policy_sample_output)[0]
        return sampled_act

    def policy_predict(self, obs):
        feed = {'obs': obs}
        means = self.fluid_executor.run(
            self.policy_predict_program,
            feed=feed,
            fetch_list=self.policy_predict_output)[0]
        return means

    def value_predict(self, obs):
        feed = {'obs': obs}
        value = self.fluid_executor.run(
            self.value_predict_program,
            feed=feed,
            fetch_list=self.value_predict_output)[0]
        return value

    def _batch_policy_learn(self, obs, actions, advantages):
        if self.loss_type == 'KLPEN':
            feed = {
                'obs': obs,
                'actions': actions,
                'advantages': advantages,
                'beta': self.beta
            }
        else:
            feed = {'obs': obs, 'actions': actions, 'advantages': advantages}
        [loss, kl] = self.fluid_executor.run(
            self.policy_learn_program,
            feed=feed,
            fetch_list=self.policy_learn_output)
        return loss, kl

    def _batch_value_learn(self, obs, val):
        feed = {'obs': obs, 'val': val}
        value_loss = self.fluid_executor.run(
            self.value_learn_program,
            feed=feed,
            fetch_list=self.value_learn_output)[0]
        return value_loss

    def policy_learn(self, obs, actions, advantages):
        """ Learn policy:
        1. Sync parameters of policy model to old policy model
        2. Fix old policy model, and learn policy model multi times
        3. if use KLPEN loss, Adjust kl loss coefficient: beta
        """
        self.alg.sync_old_policy()

        all_loss, all_kl = [], []
        for _ in range(self.policy_learn_times):
            loss, kl = self._batch_policy_learn(obs, actions, advantages)
            all_loss.append(loss)
            all_kl.append(kl)

        if self.loss_type == 'KLPEN':
            # Adative KL penalty coefficient
            if kl > self.kl_targ * 2:
                self.beta = 1.5 * self.beta
            elif kl < self.kl_targ / 2:
                self.beta = self.beta / 1.5

        return np.mean(all_loss), np.mean(all_kl)

    def value_learn(self, obs, value):
        """ Fit model to current data batch + previous data batch
        """
        data_size = obs.shape[0]

        if self.value_learn_buffer is None:
            obs_train, value_train = obs, value
        else:
            obs_train = np.concatenate([obs, self.value_learn_buffer[0]])
            value_train = np.concatenate([value, self.value_learn_buffer[1]])
        self.value_learn_buffer = (obs, value)

        all_loss = []
        for _ in range(self.value_learn_times):
            random_ids = np.arange(obs_train.shape[0])
            np.random.shuffle(random_ids)
            shuffle_obs_train = obs_train[random_ids]
            shuffle_value_train = value_train[random_ids]
            start = 0
            while start < data_size:
                end = start + self.value_batch_size
                value_loss = self._batch_value_learn(
                    shuffle_obs_train[start:end, :],
                    shuffle_value_train[start:end])
                all_loss.append(value_loss)
                start += self.value_batch_size
        return np.mean(all_loss)

In [9]:
import numpy as np
import scipy.signal

__all__ = ['Scaler']


class Scaler(object):
    """ Generate scale and offset based on running mean and stddev along axis=0
        offset = running mean
        scale = 1 / (stddev + 0.1) / 3 (i.e. 3x stddev = +/- 1.0)
    """

    def __init__(self, obs_dim):
        """
        Args:
            obs_dim: dimension of axis=1
        """
        self.vars = np.zeros(obs_dim)
        self.means = np.zeros(obs_dim)
        self.cnt = 0
        self.first_pass = True

    def update(self, x):
        """ Update running mean and variance (this is an exact method)
        Args:
            x: NumPy array, shape = (N, obs_dim)
        see: https://stats.stackexchange.com/questions/43159/how-to-calculate-pooled-
               variance-of-two-groups-given-known-group-variances-mean
        """
        if self.first_pass:
            self.means = np.mean(x, axis=0)
            self.vars = np.var(x, axis=0)
            self.cnt = x.shape[0]
            self.first_pass = False
        else:
            n = x.shape[0]
            new_data_var = np.var(x, axis=0)
            new_data_mean = np.mean(x, axis=0)
            new_data_mean_sq = np.square(new_data_mean)
            new_means = (
                (self.means * self.cnt) + (new_data_mean * n)) / (self.cnt + n)
            self.vars = (((self.cnt * (self.vars + np.square(self.means))) +
                          (n * (new_data_var + new_data_mean_sq))) /
                         (self.cnt + n) - np.square(new_means))
            self.vars = np.maximum(
                0.0, self.vars)  # occasionally goes negative, clip
            self.means = new_means
            self.cnt += n

    def get(self):
        """ returns 2-tuple: (scale, offset) """
        return 1 / (np.sqrt(self.vars) + 0.1) / 3, self.means

In [10]:
import argparse
import gym
import numpy as np
import parl
from parl.utils import logger, action_mapping
from parl.utils.rl_utils import calc_gae, calc_discount_sum_rewards


def run_train_episode(env, agent, scaler):
    obs = env.reset()
    observes, actions, rewards, unscaled_obs = [], [], [], []
    step = 0.0
    scale, offset = scaler.get()
    scale[-1] = 1.0  # don't scale time step feature
    offset[-1] = 0.0  # don't offset time step feature
    env.reset()
    while True:
        obs = obs.reshape((1, -1))
        obs = np.append(obs, [[step]], axis=1)  # add time step feature
        unscaled_obs.append(obs)
        obs = (obs - offset) * scale  # center and scale observations
        obs = obs.astype('float32')
        observes.append(obs)

        action = agent.policy_sample(obs)
        action = np.clip(action, -1.0, 1.0)
        action = action_mapping(action, env.action_space.low[0],
                                env.action_space.high[0])

        action = action.reshape((1, -1)).astype('float32')
        env.render()
        actions.append(action)

        obs, reward, done, _ = env.step(np.squeeze(action))
        rewards.append(reward)
        step += 1e-3  # increment time step feature

        if done:
            break

    return (np.concatenate(observes), np.concatenate(actions),
            np.array(rewards, dtype='float32'), np.concatenate(unscaled_obs))


def run_evaluate_episode(env, agent, scaler):
    obs = env.reset()
    rewards = []
    step = 0.0
    scale, offset = scaler.get()
    scale[-1] = 1.0  # don't scale time step feature
    offset[-1] = 0.0  # don't offset time step feature
    while True:
        obs = obs.reshape((1, -1))
        obs = np.append(obs, [[step]], axis=1)  # add time step feature
        obs = (obs - offset) * scale  # center and scale observations
        obs = obs.astype('float32')

        action = agent.policy_predict(obs)
        action = action_mapping(action, env.action_space.low[0],
                                env.action_space.high[0])

        obs, reward, done, _ = env.step(np.squeeze(action))
        env.render()
        rewards.append(reward)

        step += 1e-3  # increment time step feature

        if done:
            break
    return np.sum(rewards)


def collect_trajectories(env, agent, scaler, episodes):
    trajectories, all_unscaled_obs = [], []
    for e in range(episodes):
        obs, actions, rewards, unscaled_obs = run_train_episode(
            env, agent, scaler)
        trajectories.append({
            'obs': obs,
            'actions': actions,
            'rewards': rewards,
        })
        all_unscaled_obs.append(unscaled_obs)
    # update running statistics for scaling observations
    scaler.update(np.concatenate(all_unscaled_obs))
    return trajectories


def build_train_data(trajectories, agent):
    train_obs, train_actions, train_advantages, train_discount_sum_rewards = [], [], [], []
    for trajectory in trajectories:
        pred_values = agent.value_predict(trajectory['obs'])

        # scale rewards
        scale_rewards = trajectory['rewards'] * (1 - 0.995)

        discount_sum_rewards = calc_discount_sum_rewards(
            scale_rewards, 0.995).astype('float32')

        advantages = calc_gae(scale_rewards, pred_values, 0, 0.995,
                              0.98)

        # normalize advantages
        advantages = (advantages - advantages.mean()) / (
            advantages.std() + 1e-6)
        advantages = advantages.astype('float32')

        train_obs.append(trajectory['obs'])
        train_actions.append(trajectory['actions'])
        train_advantages.append(advantages)
        train_discount_sum_rewards.append(discount_sum_rewards)

    train_obs = np.concatenate(train_obs)
    train_actions = np.concatenate(train_actions)
    train_advantages = np.concatenate(train_advantages)
    train_discount_sum_rewards = np.concatenate(train_discount_sum_rewards)

    return train_obs, train_actions, train_advantages, train_discount_sum_rewards




In [ ]:
# 创建飞行器环境
import gym
env = gym.make("BipedalWalker-v2")
env.reset()
obs_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]
obs_dim += 1  # add 1 to obs dim for time step feature

scaler = Scaler(obs_dim)



# 根据parl框架构建agent
######################################################################
######################################################################
#
# 6. 请构建agent:  QuadrotorModel, DDPG, QuadrotorAgent三者嵌套
#
######################################################################
######################################################################

model = BipedalWalkerModel(obs_dim, act_dim, -1.0)
alg = PPO(model, act_dim=act_dim,policy_lr=ACTOR_LR,value_lr=CRITIC_LR)
agent = BiPedalwalkerAgent(alg, obs_dim, act_dim, 0.003, loss_type='CLIP')


# parl库也为DDPG算法内置了ReplayMemory，可直接从 parl.utils 引入使用
collect_trajectories(env, agent, scaler, episodes=5)

test_flag = 0
total_steps = 0
while total_steps < 100000000:
    trajectories = collect_trajectories(
        env, agent, scaler, episodes=5)
    total_steps += sum([t['obs'].shape[0] for t in trajectories])
    total_train_rewards = sum([np.sum(t['rewards']) for t in trajectories])

    train_obs, train_actions, train_advantages, train_discount_sum_rewards = build_train_data(
        trajectories, agent)

    policy_loss, kl = agent.policy_learn(train_obs, train_actions,
                                         train_advantages)
    value_loss = agent.value_learn(train_obs, train_discount_sum_rewards)

    if total_steps // TEST_EVERY_STEPS >= test_flag: # 每隔一定step数，评估一次模型
        while total_steps // TEST_EVERY_STEPS >= test_flag:
            test_flag += 1
 
        eval_reward = run_evaluate_episode(env, agent, scaler)
        logger.info('Steps {}, Evaluate reward: {}'.format(
                total_steps, eval_reward))# 打印评估的reward

        # 每评估一次，就保存一次模型，以训练的step数命名
#         ckpt = 'model_dir/steps_{}.ckpt'.format(total_steps)
#         agent.save(ckpt)



[06-28 13:33:16 MainThread @machine_info.py:88] Cannot find available GPU devices, using CPU now.
name: "obs"
type {
  type: LOD_TENSOR
  lod_tensor {
    tensor {
      data_type: FP32
      dims: -1
      dims: 25
    }
    lod_level: 0
  }
}
persistable: false

name: "fc_1.tmp_2"
type {
  type: LOD_TENSOR
  lod_tensor {
    tensor {
      data_type: FP32
      dims: -1
      dims: 4
    }
    lod_level: 0
  }
}
persistable: false
 name: "create_parameter.w_0"
type {
  type: LOD_TENSOR
  lod_tensor {
    tensor {
      data_type: FP32
      dims: 4
    }
  }
}
persistable: true

aaaaa
[06-28 13:33:16 MainThread @machine_info.py:88] Cannot find available GPU devices, using CPU now.
[06-28 13:36:09 MainThread @machine_info.py:88] Cannot find available GPU devices, using CPU now.
[06-28 13:36:11 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 1999, Evaluate reward: -96.7204967762089
[06-28 13:39:06 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 10836, Evaluate reward: -33.0

[06-28 17:44:44 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 615422, Evaluate reward: 223.49776001399385
[06-28 17:48:31 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 621904, Evaluate reward: 218.64807705426216
[06-28 17:56:06 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 637904, Evaluate reward: 206.47997165803673
[06-28 18:00:21 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 645904, Evaluate reward: 225.8231842094157
[06-28 18:04:24 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 653752, Evaluate reward: 207.51571455761604
[06-28 18:08:55 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 661752, Evaluate reward: 223.08589108535955
[06-28 18:15:51 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 676236, Evaluate reward: 225.00745830753942
[06-28 18:19:56 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 684236, Evaluate reward: 236.4745701614848
[06-28 18:24:17 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 692236, Evaluate rew

[06-28 23:29:28 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 1286306, Evaluate reward: 246.978987059148
[06-28 23:33:17 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 1294306, Evaluate reward: 251.42812024933164
[06-28 23:36:58 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 1302306, Evaluate reward: 230.75237336429393
[06-28 23:40:44 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 1310306, Evaluate reward: 242.91636031022665
[06-28 23:47:51 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 1326306, Evaluate reward: 219.8094091451876
[06-28 23:51:47 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 1334306, Evaluate reward: 234.6478143842537
[06-28 23:55:33 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 1342306, Evaluate reward: 244.91088921142554
[06-28 23:59:20 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 1350271, Evaluate reward: 238.58095465933346
[06-29 00:06:08 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 1365964, Evalu

[06-29 04:25:45 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 1950241, Evaluate reward: 219.7972831671654
[06-29 04:32:40 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 1966241, Evaluate reward: 234.97775551762248
[06-29 04:36:26 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 1974241, Evaluate reward: 215.33524045896155
[06-29 04:39:57 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 1981739, Evaluate reward: 112.19274581854543
[06-29 04:46:48 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 1997595, Evaluate reward: 267.26331055185074
[06-29 04:50:31 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 2005539, Evaluate reward: 268.63409972513904
[06-29 04:54:06 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 2013155, Evaluate reward: 263.0783014684369
[06-29 04:57:51 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 2021155, Evaluate reward: 260.1788816907574
[06-29 05:04:46 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 2037155, Eval

[06-29 09:28:57 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 2621274, Evaluate reward: 245.3858790047827
[06-29 09:35:25 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 2635883, Evaluate reward: 247.54709935103853
[06-29 09:39:45 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 2643843, Evaluate reward: 246.144580896991
[06-29 09:44:28 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 2651843, Evaluate reward: 243.93687082114505
[06-29 09:52:57 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 2667576, Evaluate reward: 240.33544183875247
[06-29 09:57:38 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 2675576, Evaluate reward: 241.72994440850678
[06-29 10:02:23 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 2683576, Evaluate reward: 242.31045636204323
[06-29 10:06:37 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 2691075, Evaluate reward: 244.28606957996757
[06-29 10:13:19 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 2707075, Eval

[06-29 15:22:44 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 3294612, Evaluate reward: 268.7454922301596
[06-29 15:27:19 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 3302612, Evaluate reward: 259.2924377625777
[06-29 15:33:18 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 3310612, Evaluate reward: 268.1642251443261
[06-29 15:41:36 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 3323321, Evaluate reward: 257.59219513471237
[06-29 15:45:49 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 3330137, Evaluate reward: 39.98526110406592
[06-29 15:54:39 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 3343910, Evaluate reward: 144.14881027385653
[06-29 15:59:28 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 3351043, Evaluate reward: 254.60236954655932
[06-29 16:10:52 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 3364967, Evaluate reward: 262.8042495616227
[06-29 16:20:14 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 3372731, Evalua

[06-29 23:08:14 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 3965272, Evaluate reward: 262.5328613665414
[06-29 23:12:27 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 3972570, Evaluate reward: 263.81184196158813
[06-29 23:19:50 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 3986418, Evaluate reward: 265.72438235487664
[06-29 23:24:06 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 3993906, Evaluate reward: 267.9550156132989
[06-29 23:30:10 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 4005381, Evaluate reward: 272.0535110037259
[06-29 23:34:10 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 4012344, Evaluate reward: 263.42604312723626
[06-29 23:40:18 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 4023942, Evaluate reward: 270.1162593898314
[06-29 23:43:50 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 4030058, Evaluate reward: 267.655414180673
[06-29 23:50:27 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 4042606, Evaluat

[06-30 05:09:20 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 4634609, Evaluate reward: 272.9855363217468
[06-30 05:13:12 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 4641587, Evaluate reward: 269.59091266858263
[06-30 05:20:24 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 4655600, Evaluate reward: 271.0869479456587
[06-30 05:24:11 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 4662436, Evaluate reward: 272.5511168181399
[06-30 05:30:59 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 4675881, Evaluate reward: 96.56334555957963
[06-30 05:33:41 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 4680400, Evaluate reward: 272.59603741568884
[06-30 05:40:49 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 4694634, Evaluate reward: 73.53664788946944
[06-30 05:44:39 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 4701541, Evaluate reward: 270.8883795129005
[06-30 05:50:03 MainThread @<ipython-input-11-8251e3445aae>:51] Steps 4711764, Evaluat